In [7]:
from mlflow import MlflowClient
from pprint import pprint

In [8]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

In [13]:
all_experiments = client.search_experiments()

print(all_experiments[0].name)

MLFlow-Start


In [15]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


In [17]:
experiment_description = (
    "A Test Experiment about MNIST Dataset in MLFlow"
)

experiment_tags = {
    "project-name": "Mnist-Lab",
    "mlflow.note.content": experiment_description
}

mnist_experiment = client.create_experiment(
    name="MLFlow-Start-1",
    tags=experiment_tags
)

In [18]:
# Use search_experiments() to search on the project_name tag key

mnistlab_experiment = client.search_experiments(
    filter_string="tags.`project-name` = 'Mnist-Lab'"
)


In [19]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize


transform = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))
])

data_train = MNIST(root="./", download=True, train=True, transform=transform)
data_test = MNIST(root="./", download=True, train=False, transform=transform)

In [20]:

params = {
    "lr": 0.0001,
    "epochs": 5
}

In [21]:
from torch.utils.data import DataLoader

train_loaded = DataLoader(data_train, batch_size=32, shuffle=True)
test_loaded = DataLoader(data_test, batch_size=32, shuffle=True)

In [22]:
from torch import nn

class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        self.cnn_layers = nn.Sequential(
            #CONV1
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            #CONV2
            nn.Conv2d(6, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(32*5*5, 200),
            nn.ReLU(),
            nn.Linear(200, 80),
            nn.ReLU(),
            nn.Linear(80, 10),
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x
    

model = MnistModel()

In [23]:
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=params["lr"])
criterion = nn.CrossEntropyLoss()

In [24]:
#Training the model
from tqdm import tqdm
import torch
#Lists to get all the data about training
train_loss, test_loss = [], []
accuracy_train, accuracy_test = [], []
for epoch in range(params["epochs"]):
    total_train_loss = 0
    total_test_loss = 0
    
    model.train()
    
    total = 0
    for index, (image, label) in tqdm(enumerate(train_loaded), desc=f"Fitting Epoch {epoch + 1}"):
        
        optimizer.zero_grad()
        
        pred = model(image)
        
        loss = criterion(pred, label)
        total_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    
    train_accuracy = total / len(data_train)
    total_train_loss = total_train_loss / (index + 1)
    
    accuracy_train.append(train_accuracy)
    train_loss.append(total_train_loss)
    
    #Validating the model
    model.eval()
    total = 0
    for index, (image, label) in tqdm(enumerate(test_loaded), desc="Validating the model"):
        pred = model(image)
        
        loss = criterion(pred, label)
        total_test_loss += loss.item()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    test_accuracy = total / len(data_test)
    total_test_loss = total_test_loss / (index + 1)
    
    accuracy_test.append(test_accuracy)
    test_loss.append(total_test_loss)
    
    print("Epoch: {}/{}  ".format(epoch + 1, params["epochs"]),
            "Training loss: {:.4f}  ".format(total_train_loss),
            "Testing loss: {:.4f}  ".format(total_test_loss),
            "Train accuracy: {:.4f}  ".format(train_accuracy),
            "Test accuracy: {:.4f}  ".format(test_accuracy))

Fitting Epoch 1: 0it [00:00, ?it/s]

Fitting Epoch 1: 1875it [00:26, 70.80it/s]
Validating the model: 313it [00:03, 102.83it/s]


Epoch: 1/5   Training loss: 0.5134   Testing loss: 0.1806   Train accuracy: 0.8535   Test accuracy: 0.9481  


Fitting Epoch 2: 1875it [00:29, 63.28it/s]
Validating the model: 313it [00:03, 90.91it/s]


Epoch: 2/5   Training loss: 0.1528   Testing loss: 0.1144   Train accuracy: 0.9542   Test accuracy: 0.9663  


...

Epoch: 3/5   Training loss: 0.1060   Testing loss: 0.0817   Train accuracy: 0.9679   Test accuracy: 0.9747  


Fitting Epoch 4: 1875it [00:30, 61.13it/s]
Validating the model: 313it [00:03, 90.33it/s] 


Epoch: 4/5   Training loss: 0.0847   Testing loss: 0.0691   Train accuracy: 0.9740   Test accuracy: 0.9774  


Fitting Epoch 5: 1875it [00:31, 60.09it/s]
Validating the model: 313it [00:03, 93.93it/s] 

Epoch: 5/5   Training loss: 0.0721   Testing loss: 0.0655   Train accuracy: 0.9781   Test accuracy: 0.9791  


In [25]:
metrics = {
    "accuracy-average-train": sum(accuracy_train) / len(accuracy_train),
    "accuracy-average-test": sum(accuracy_test) / len(accuracy_test),
    "loss-average-train": sum(train_loss) / len(train_loss),
    "loss-average-test": sum(test_loss) / len(test_loss)
}

In [26]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")

experiment = {
    "name": "MLFlow-Start",
    "run-name": "mnist_test",
    "artifact_path": "rf_mnist"
}

mlflow.set_experiment(experiment["name"])

with mlflow.start_run(run_name=experiment["run-name"]) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.pytorch.log_model(
        pytorch_model = model,
        artifact_path = experiment["artifact_path"]
    )
    

2024/06/19 17:38:11 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.2.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/06/19 17:38:16 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.2.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
